In [1]:
import pandas as pd
import networkx as nx
import numpy as np

In [2]:
filename = 'edges karate.csv'
#filename = 'edges lesmis.csv'
edges = pd.read_csv(filename, sep=';', header=None, names=['id1', 'id2', 'weight'])
edges.head()

,id1,id2,weight
0,2,1,1
1,3,1,1
2,3,2,1
3,4,1,1
4,4,2,1


In [3]:
edges.head()

,id1,id2,weight
0,2,1,1
1,3,1,1
2,3,2,1
3,4,1,1
4,4,2,1


In [4]:
G = nx.from_pandas_edgelist(edges, 'id1', 'id2', edge_attr=True)
G = G.to_directed() # E[1, 2] and E[2, 1] must be different references

In [5]:
G.nodes()

NodeView((2, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17, 18, 20, 22, 26, 24, 25, 28, 29, 30, 27, 31, 32, 33, 15, 16, 19, 21, 23, 34))

In [6]:
G.edges()

OutEdgeView([(2, 1), (2, 3), (2, 4), (2, 8), (2, 14), (2, 18), (2, 20), (2, 22), (2, 31), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 11), (1, 12), (1, 13), (1, 14), (1, 18), (1, 20), (1, 22), (1, 32), (3, 1), (3, 2), (3, 4), (3, 8), (3, 9), (3, 10), (3, 14), (3, 28), (3, 29), (3, 33), (4, 1), (4, 2), (4, 3), (4, 8), (4, 13), (4, 14), (5, 1), (5, 7), (5, 11), (6, 1), (6, 7), (6, 11), (6, 17), (7, 1), (7, 5), (7, 6), (7, 17), (8, 1), (8, 2), (8, 3), (8, 4), (9, 1), (9, 3), (9, 31), (9, 33), (9, 34), (10, 3), (10, 34), (11, 1), (11, 5), (11, 6), (12, 1), (13, 1), (13, 4), (14, 1), (14, 2), (14, 3), (14, 4), (14, 34), (17, 6), (17, 7), (18, 1), (18, 2), (20, 1), (20, 2), (20, 34), (22, 1), (22, 2), (26, 24), (26, 25), (26, 32), (24, 26), (24, 28), (24, 30), (24, 33), (24, 34), (25, 26), (25, 28), (25, 32), (28, 3), (28, 24), (28, 25), (28, 34), (29, 3), (29, 32), (29, 34), (30, 24), (30, 27), (30, 33), (30, 34), (27, 30), (27, 34), (31, 2), (31, 9), (31, 33), (31, 

In [7]:
def get_weight(G, u, v):
    return G.get_edge_data(u, v)['weight']

def dependency_coefficient(G, x, v, y):
    return get_weight(G, v, y) / (get_weight(G, x, v) + get_weight(G, v, y))

for x, y in G.edges():
    edge = G.get_edge_data(x, y)
    w = edge['weight']
    N = set(G.neighbors(x))
    #CN = nx.common_neighbors(G, x, y) # not implemented for directed type
    CN = N & set(G.neighbors(y))
    
    edge['D'] = (w + sum(get_weight(G, x, v) * dependency_coefficient(G, x, v, y) for v in CN)) / sum(get_weight(G, x, v) for v in N)

In [8]:
filename_verification = 'edges_dep_directed karate.csv'
#filename_verification = 'edges_dep_directed lesmis.csv'
edges_verification = pd.read_csv(filename_verification, sep=';', header=None, names=['id1', 'id2', 'dep verification'])
edges_verification.head()

,id1,id2,dep verification
0,2,1,0.50000
1,1,2,0.28125
2,3,1,0.35000
3,1,3,0.21875
4,3,2,0.30000


In [9]:
edges_verification['dep'] = edges_verification.apply(lambda row: G.get_edge_data(row['id1'], row['id2'])['D'], axis=1)
edges_verification.head()

,id1,id2,dep verification,dep
0,2,1,0.50000,0.50000
1,1,2,0.28125,0.28125
2,3,1,0.35000,0.35000
3,1,3,0.21875,0.21875
4,3,2,0.30000,0.30000


In [10]:
np.testing.assert_allclose(edges_verification['dep'], edges_verification['dep verification'])

In [11]:
edges_verification.to_csv('verification' + filename_verification)

In [12]:
def is_dependent(G, x, y, threshold = 0.5):
    return G.get_edge_data(x,y)['D'] >= threshold

In [13]:
def zone_detection(G, ego):
    zone = set()
    inner_zone = set()
    outer_zone = set()
    
    # detection of inner-zone
    nodes_last_added = set()
    nodes_outside = set()
    inner_zone.add(ego)
    nodes_last_added.add(ego)
    
    while len(nodes_last_added) > 0:
        for v in nodes_last_added:
            for adj in G.neighbors(v):
                if adj not in inner_zone and is_dependent(G, adj, v):
                    nodes_outside.add(adj)
        inner_zone.update(nodes_outside)
        nodes_last_added.clear()
        nodes_last_added.update(nodes_outside)
        nodes_outside.clear()
    
    # detection of outer-zone
    for v in inner_zone:
        for adj in G.neighbors(v):
            if is_dependent(G, v, adj) and adj not in inner_zone:
                outer_zone.add(adj)

    # composition of zone
    zone.update(inner_zone)
    zone.update(outer_zone)
    
    return zone, inner_zone, outer_zone

In [14]:
assert zone_detection(G, 1)[0] == set([14,4,8,2,20,18,22,1,13,12,5,11,7,6,17,3])
assert zone_detection(G, 34)[0] == set([34,32,3,24,30,27,15,16,21,33,23,19,31,9,10,29])
assert zone_detection(G, 6)[0] == set([1,5,11,7,6,17])
assert zone_detection(G, 7)[0] == set([1,5,11,7,6,17])
assert zone_detection(G, 33)[0] == set([33,30,27,15,16,21,23,19,31,9,34,24])
assert zone_detection(G, 2)[0] == set([2,20,22,18,8,14,4,13,1,3])
assert zone_detection(G, 30)[0] == set([30,27,24,33,34])
assert zone_detection(G, 24)[0] == set([30,27,24,33,34])
assert zone_detection(G, 4)[0] == set([4,14,8,13,2,3,1])
assert zone_detection(G, 9)[0] == set([9,31,33,34])
assert zone_detection(G, 31)[0] == set([9,31,33,34])
assert zone_detection(G, 3)[0] == set([3,34,10,14,4,8,13,1,2])
assert zone_detection(G, 32)[0] == set([32,34,26,25,29])

assert zone_detection(G, 5)[0] == set([5,11,7,6,1])
assert zone_detection(G, 11)[0] == set([5,11,7,6,1])
assert zone_detection(G, 8)[0] == set([8,4,3,2,1])
assert zone_detection(G, 17)[0] == set([17,7,6])
assert zone_detection(G, 14)[0] == set([14,3,4,2,1])
assert zone_detection(G, 25)[0] == set([25,26,32])
assert zone_detection(G, 26)[0] == set([25,26,32])
assert zone_detection(G, 10)[0] == set([10,34,3])

assert zone_detection(G, 12)[0] == set([12,1])
assert zone_detection(G, 13)[0] == set([13,1,4])
assert zone_detection(G, 15)[0] == set([15,33,34])
assert zone_detection(G, 16)[0] == set([16,33,34])
assert zone_detection(G, 18)[0] == set([18,1,2])
assert zone_detection(G, 19)[0] == set([19,33,34])
assert zone_detection(G, 20)[0] == set([20,1,2])
assert zone_detection(G, 21)[0] == set([21,33,34])
assert zone_detection(G, 22)[0] == set([22,1,2])
assert zone_detection(G, 23)[0] == set([23,33,34])
assert zone_detection(G, 27)[0] == set([27,30,34])
assert zone_detection(G, 28)[0] == set([28])
assert zone_detection(G, 29)[0] == set([29,32,34])